# Example of usage of Cross Validation while mixing models
Not well implemented yet


In [1]:
sc

# Import

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100


In [3]:
from pyspark.sql.functions import col

In [4]:
from cross_validation import KFoldIndexes,CrossValidation

In [5]:
from cross_validation_blending import CrossValidationBlending

In [6]:
from als import predictions_ALS
from means import user_mean,global_mean,movie_mean

In [7]:
import helpers

In [8]:
import random


# Dataframe creation

In [9]:
train = helpers.load_csv()
train.head()

,User,Movie,Rating
0,44,1,4
1,61,1,3
2,67,1,4
3,72,1,3
4,86,1,5


# Cross Validation Blending examples
Example on how CrossValidationBlending

In [64]:
blending_test=CrossValidationBlending(train,4)

In [65]:
blending_test.add_model(predictions_ALS,'als')
blending_test.add_model(predictions_ALS,'als2')

In [66]:
blending_test.add_model(global_mean,'global_mean')

In [67]:
blending_test.add_model(user_mean,'user_mean')

In [68]:
blending_test.add_model(movie_mean,'movie_mean')

In [69]:
blending_test.add_params_for_model('als',{'spark_context':sc,'rank':8,'lambda_':0.081, 'iterations':24, 'nonnegative':True},compute_predictions=True)
blending_test.add_params_for_model('als2',{'spark_context':sc,'rank':8,'lambda_':0.081, 'iterations':24, 'nonnegative':True},compute_predictions=True)

[ALS] applying
[ALS] done
[ALS] applying
[ALS] done
[ALS] applying
[ALS] done
[ALS] applying
[ALS] done
[ALS] applying
[ALS] done
[ALS] applying
[ALS] done
[ALS] applying
[ALS] done
[ALS] applying
[ALS] done


In [97]:
blending_test.add_params_for_model('user_mean',{})

[USER_MEAN] applying
[USER_MEAN] done
[USER_MEAN] applying
[USER_MEAN] done
[USER_MEAN] applying
[USER_MEAN] done
[USER_MEAN] applying
[USER_MEAN] done


In [104]:
blending_test.add_params_for_model('movie_mean',{})

[MOVIE_MEAN] applying
Movie
1     3.391304
2     3.525799
3     3.511745
4     3.947524
5     3.584728
6     4.680365
7     3.455934
8     3.877417
9     2.922680
10    3.652151
Name: Rating, dtype: float64
1 3.39130434783
1 3.39130434783
2 3.5257985258
3 3.51174496644
4 3.94752433347
5 3.58472803347
6 4.6803652968
7 3.45593419506
8 3.87741686002
9 2.92268041237
10 3.65215137377
11 3.26767091541
12 3.21179624665
13 3.50154798762
14 4.42765382015
15 3.1312849162
16 3.4501607717
17 2.66417910448
18 2.41811175337
19 3.45940594059
20 2.92448759439
21 3.58618181818
22 3.09989909183
23 3.35660377358
24 3.71546391753
25 3.56053811659
26 4.13989361702
27 3.725
28 3.68765743073
29 3.5365497076
30 3.88215488215
31 3.22325581395
32 3.4296005239
33 3.40596330275
34 3.41061452514
35 3.10942956927
36 3.43127962085
37 3.73208722741
38 3.06331471136
39 3.03875968992
40 3.7539766702
41 2.88888888889
42 3.48053892216
43 3.59646017699
44 3.5931254996
45 4.04637096774
46 4.46017137417
47 3.42307692308
48 

In [90]:
blending_test.add_params_for_model('global_mean',{},compute_predictions=True)

[GLOBAL_MEAN] applying
3.8574917810298692
[GLOBAL_MEAN] done
[GLOBAL_MEAN] applying
3.8568879614461764
[GLOBAL_MEAN] done
[GLOBAL_MEAN] applying
3.856957066501721
[GLOBAL_MEAN] done
[GLOBAL_MEAN] applying
3.857785194298493
[GLOBAL_MEAN] done


In [101]:
blending_test.evaluate_blending({'als':0,'als2':0,'global_mean':1,'user_mean':0,'movie_mean':0})

1.1190571700995784

In [81]:
best_dict={'als':0.4,'als2':0.4,'global_mean':0,'user_mean':0.05,'movie_mean':0.15}

In [48]:
blending_test.models

{'als': <function als.predictions_ALS>,
 'als2': <function als.predictions_ALS>,
 'global_mean': <function means.global_mean>}

# Test with real predictions

In [82]:
test = pd.read_csv('../data/sampleSubmission.csv')
test.head()

,Id,Prediction
0,r37_c1,3
1,r73_c1,3
2,r156_c1,3
3,r160_c1,3
4,r248_c1,3


In [83]:
# Prepare test for RDD
test_prep = test
test_prep['User'] = test_prep['Id'].apply(lambda x: int(x.split('_')[0][1:]))
test_prep['Movie'] = test_prep['Id'].apply(lambda x: int(x.split('_')[1][1:]))
test_prep['Rating'] = test_prep['Prediction']
test_prep = test_prep.drop(['Prediction', 'Id'], axis=1)
test_prep.head()

,User,Movie,Rating
0,37,1,3
1,73,1,3
2,156,1,3
3,160,1,3
4,248,1,3


In [85]:
pred=blending_test.evaluate_blending_for_validation(best_dict,train,test_prep)

{'nonnegative': True, 'spark_context': <pyspark.context.SparkContext object at 0x7fea8c7e1208>, 'lambda_': 0.081, 'iterations': 24, 'rank': 8}
[ALS] applying
[ALS] done
{}
[MOVIE_MEAN] applying
[MOVIE_MEAN] done
{}
[USER_MEAN] applying
[USER_MEAN] done
{}
[GLOBAL_MEAN] applying
[GLOBAL_MEAN] done
{'nonnegative': True, 'spark_context': <pyspark.context.SparkContext object at 0x7fea8c7e1208>, 'lambda_': 0.081, 'iterations': 24, 'rank': 8}
[ALS] applying
[ALS] done


In [86]:
test.Rating=pred
test['Prediction']=test.Rating

In [87]:
test.head()

,Id,Prediction,User,Movie,Rating
0,r37_c1,3.244184,37,1,3.244184
1,r73_c1,3.123454,73,1,3.123454
2,r156_c1,3.673083,156,1,3.673083
3,r160_c1,3.305094,160,1,3.305094
4,r248_c1,3.464529,248,1,3.464529


In [88]:
test = test.drop(['User', 'Movie', 'Rating'], axis=1)

In [89]:
test.to_csv('pred_pyspark_als.csv', index=False)

# Test with a general prediction function
Example of usage of a function - for all function the usage should be the same

In [13]:
train_index=random.sample(range(1176952),1000000)
train_index.sort()
test_index=list(set(range(1176952))-set(train_index))
test_index.sort()

In [14]:
training=train.loc[train_index]
testing=train.loc[test_index]

In [ ]:
predictions_ALS(training,testing,sc,rank=8,lambda_=0.081, iterations=24, nonnegative=True).head()

# Usage of simple Cross Validation on a single model


# Models

In [ ]:
class ALSModel:
    
    def __init__(self):
        pass
    
    def fit(self,data,**arg):
        self.model = ALS.train(data, **arg)
    
    def predict(self,data):
        data_for_preditions=data.map(lambda x: (x[0], x[1]))
        self.predictions = self.model.predictAll(data_for_preditions).map(lambda r: ((r[0], r[1]), r[2]))
    
    def evaluate(self,data):
        rates_and_preds = data.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(self.predictions)
        error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        return error

# Cross validation

In [ ]:
cv=CrossValidation(train,4,True,sc)

In [ ]:
ranks = [8]
lambdas = [0.08,0.081,0.082,0.083,0.084,0.085,0.086,0.087,0.088,0.089,0.09]
numIters = [24]
nbr_models = len(ranks)*len(lambdas)*len(numIters)

bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [ ]:
bestLambda

In [ ]:
i = 0
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    try:
        print(rank,lmbda,numIter)
        validationRmse = cv.evaluate(ALSModel(),rank=rank,lambda_=lmbda, iterations=numIter, nonnegative=True)
        validationRmse = np.mean(validationRmse)
        print("Model %i/%i: RMSE (validation) = %f" %(i+1, nbr_models, validationRmse))
        print("  Trained with rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))
        print("")
        if (validationRmse < bestValidationRmse):
#             bestModel = model
            bestValidationRmse = validationRmse
            bestRank = rank
            bestLambda = lmbda
            bestNumIter = numIter
    except:
        print("Model %i/%i failed!" %(i+1, nbr_models))
        print("  Parameters: rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))

    i += 1
    
# Evaluate the best model on the training set
print("The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) \
  + "and numIter = %d, and its RMSE on the training set is %f" % (bestNumIter, bestValidationRmse))

# # Evaluate the best model on the test set
# testRmse = computeRMSE(bestModel, test_for_predict_RDD, test_RDD)
# print("RMSE on the test set: %f"%(testRmse))


# Prepare data for submission

In [ ]:
test = pd.read_csv('../data/sampleSubmission.csv')
test.head()

In [ ]:
# Prepare test for RDD
test_prep = test
test_prep['UserID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[0][1:]))
test_prep['MovieID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[1][1:]))
test_prep['Rating'] = test_prep['Prediction']
test_prep = test_prep.drop(['Prediction', 'Id'], axis=1)
test_prep.head()

In [ ]:
# First, we transform it using sqlContect
test_sql = sqlContext.createDataFrame(test_prep)
test_rdd = test_sql.rdd
test_rdd.take(3)

In [ ]:
bestModel=ALSModel()
bestModel.fit(train_rdd,rank=8,lambda_=0.081, iterations=24, nonnegative=True)

In [ ]:
bestModel.predict(test_rdd)
predictions=bestModel.predictions
# predictions = bestModel.predictAll(test_RDD_Kaggle).map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
predictions.take(3)

In [ ]:
pred_df = predictions.toDF().toPandas()

In [ ]:
pred_df['UserID'] = pred_df['_1'].apply(lambda x: x['_1'])
pred_df['MovieID'] = pred_df['_1'].apply(lambda x: x['_2'])
pred_df['Rating'] = pred_df['_2']
pred_df = pred_df.drop(['_1', '_2'], axis=1)
pred_df.head()

In [ ]:
pred_df = pred_df.sort_values(by=['MovieID', 'UserID'])
pred_df.head()

In [ ]:
pred_df.index = range(len(pred_df))

In [ ]:
test['Prediction'] = pred_df['Rating']


In [ ]:
test.head()

In [ ]:
test = test.drop(['UserID', 'MovieID', 'Rating'], axis=1)


In [ ]:
test.to_csv('pred_pyspark_als.csv', index=False)